# 病院分布データを作成

- 元データ：https://kouseikyoku.mhlw.go.jp/kantoshinetsu/chousa/shitei.html

`081shitei_ika_ibaraki_r0309.xlsx`は、上記URLから取得したPDFをExcelに変換したもの。Githubには置いていません。

In [ ]:
import numpy as np
import pandas as pd

prefecture = 'ibaraki'
df = pd.read_excel('dataset/ibaraki/081shitei_ika_ibaraki_r0309.xlsx', index_col=0)

In [ ]:
### 郵便番号から緯度経度
import requests
import json
import time

def zipcode_to_loc(zipcode):
    url = 'http://geoapi.heartrails.com/api/json'
    payload = {'method':'searchByPostal'}
    payload['postal']= zipcode

    time.sleep(3)
    
    if 'location' in requests.get(url, params=payload).json()['response']:
        res = requests.get(url, params=payload).json()['response']['location'][0]
        return res['y'], res['x']
    else:
        return ('0','0')
    

In [ ]:
# ちゃんと動いているか確認する用
# df = df.head(10).copy()

In [ ]:
df_locations = pd.DataFrame( columns=['name','postal_code','latitude','longitude','num_doctors'] )
cnt = 0
for i,r in df.iterrows():
    r_member = r['電話番号\n／ 勤務医数']
    if pd.isnull(r_member) == False:
        if len(r_member.split('\n')) > 1:
            if r_member.split('\n')[1][-1] != '数':
                if r_member.split('\n')[1][-1] == ')':
                    num_doctors = int(r_member.split('\n')[1][-2])
                else:
                    num_doctors = int(r_member.split('\n')[1][-1])
    
                if num_doctors > 0:
                    name = r['医 療 機 関 名 称']
                    location = r['医療機関所在地']
                    if "\n" in str(location):
                        location_arr = location.split('\n')
                        zipcode = location_arr[0][1:]
                        zipcode = zipcode.translate(str.maketrans({chr(0xFF01 + i): chr(0x21 + i) for i in range(94)}))
                        loc = zipcode_to_loc(zipcode)
                        tmp_se = pd.Series( [ name, zipcode, float(loc[0]), float(loc[1]), num_doctors ], index=df_locations.columns )
                        df_locations = df_locations.append( tmp_se, ignore_index=True )
                        cnt += 1
                        print(cnt)


In [ ]:
df_locations = df_locations[~(df_locations['latitude'] == 0)].copy()
df_locations

In [ ]:
df_locations.to_excel('dataset/'+prefecture+'/hospitals_'+prefecture+'.xlsx')

# Generate a map

In [ ]:
import folium
 
map = folium.Map(location=[36.375394,140.477223], zoom_start=14)

for i, r in df_locations.iterrows():
    folium.Marker(location=[r['latitude'], r['longitude']], popup=r['name']).add_to(map)
    
map.save("results/"+prefecture+"/map_hospitals_"+prefecture+".html")